In [3]:
import pandas as pd
import h3
import folium

In [4]:
df = pd.read_csv("bronze\merged_sopp.csv")
display(df[["Østkoordinat", "Nordkoordinat"]])

C:\Users\mafl9\AppData\Local\Temp\ipykernel_1872\1778571956.py:1: DtypeWarning: Columns (12,13,14,23,24,25,27,29,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("bronze\merged_sopp.csv")


,Østkoordinat,Nordkoordinat
0,10.830053,59.863743
1,10.822157,59.872112
2,12.251898,64.023058
3,12.246454,64.018069
4,12.246669,64.020209
...,...,...
591366,10.825651,59.460626
591367,10.825651,59.460626
591368,10.828194,59.461941
591369,10.828391,59.462383


In [5]:
def convert_to_h3(row):
    h3_cell = h3.geo_to_h3(row['Østkoordinat'], row['Nordkoordinat'],4) 
    return h3_cell

In [6]:
df['h3_cell'] = df.apply(convert_to_h3, axis=1)

In [7]:
display(df['h3_cell'])

0         84622e3ffffffff
1         84622e3ffffffff
2         8463411ffffffff
3         8463411ffffffff
4         8463411ffffffff
               ...       
591366    84622e7ffffffff
591367    84622e7ffffffff
591368    84622e7ffffffff
591369    84622e7ffffffff
591370    84622e7ffffffff
Name: h3_cell, Length: 591371, dtype: object

In [8]:
df['h3_cell'].nunique()

691

In [9]:
map_center = [df['Nordkoordinat'].mean(), df['Østkoordinat'].mean()]
mymap = folium.Map(location=map_center, zoom_start=10)

In [10]:
# Modify the loop for creating GeoJSON features
geojson_features = []
for h3_cell in df['h3_cell'].unique():
    hexagon = h3.h3_to_geo_boundary(h3_cell, geo_json=True)
    coordinates = [[coord[1], coord[0]] for coord in hexagon] 
    feature = {
        'type': 'Feature',
        'properties': {'h3_cell': h3_cell},
        'geometry': {
            'type': 'Polygon',
            'coordinates': [coordinates]
        }
    }
    geojson_features.append(feature)

In [11]:
folium.GeoJson(
    {'type': 'FeatureCollection', 'features': geojson_features},
    name='h3_grid'
).add_to(mymap)

In [12]:
mymap.save('h3_map.html')